# 변수 추출

RNA-Seq 데이터 샘플

```
ENSG00000242268.2	0.0
ENSG00000270112.3	0.150012657236
ENSG00000167578.15	6.76359958728
ENSG00000273842.1	0.0
ENSG00000078237.5	17.398229005
ENSG00000146083.10	41.8367351424

~~~~

ENSG00000231119.2	0.073160516105
ENSG00000280861.1	0.0
ENSG00000123685.7	2.36108104824
ENSG00000181518.3	0.0
```

- http://asia.ensembl.org/index.html 에서 유전자 정보를찾아볼 수 있음.

- ENSG00000270112.3	 : 변수,  ensembl에서 사용되는 Gene ID임.
- 0.150012657236     : 변수의 값, 발현량을 정규화시킨 값    
- 변수( ENSG00000270112.3 )를  배열에서 몇번째 위치시킬 것인지에 대한 위치 정보를 mysql 테이블에 저장함.


## GeneID의 배열 위치(index)을  DB에 저장하기

MYSQL 스키마 구조 

```
use tcga_gdc;

DROP TABLE IF EXISTS gene_index CASCADE ;
CREATE TABLE IF NOT EXISTS gene_index (
  ensembl_id   VARCHAR(20)  PRIMARY KEY,
  seq_index    INT,
  use_flag     VARCHAR(1) DEFAULT 'Y'
);
```

In [ ]:
# mysql_python.py에 아래 함수 추가
def insert_gene_index(info ):
    query = "INSERT INTO gene_index(ensembl_id, seq_index) " \
            "VALUES(%s,%s)"

    try:
        conn = connect()

        cursor = conn.cursor()
        cursor.executemany(query, info)

        conn.commit()
    except Error as e:
        print('Error:', e)

    finally:
        cursor.close()
        conn.close()


In [ ]:
# insert_gene_index.py
from  mysql_python import insert_gene_index

FETCH_SIZE = 100

with open('037bf43c-7de6-4f82-80bc-8c9673fbdd7d.FPKM.txt','r') as gdc :
    cnt = 0
    in_data = []

    for line in gdc:
        cnt += 1

        #print( line )
        rows = line.strip().split('\t')
        in_data.append( ( rows[0].split('.')[0] , cnt-1  )  )

        if( cnt % FETCH_SIZE == 0) :
            insert_gene_index( in_data )
            in_data = []


    insert_gene_index( in_data )
